In [9]:
import torch

In [10]:
data = torch.load("debug_nan_4.pt")

In [11]:
data

{'current_iteration': {'model_input': tensor([[[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           ...,
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0'),
  'model_output': tensor([[[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           ...,
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0',
         requires_grad=True),
  't': tensor([0.0158], device='cuda:0'),
  'iteration': 4},
 'previous_iteration': {'model_input': tensor([[[1.6549e-42, 0.0000e+00, 5.4062e-31,  ..., 9.0163e-37,
            4.8625e-43, 1.3649e-32],
           [3.8580e-37, 1.2394e-40, 3.5159e-42,  ..., 2.42

In [12]:
previous_iteration = data["previous_iteration"]

In [13]:
previous_iteration

{'model_input': tensor([[[1.6549e-42, 0.0000e+00, 5.4062e-31,  ..., 9.0163e-37,
           4.8625e-43, 1.3649e-32],
          [3.8580e-37, 1.2394e-40, 3.5159e-42,  ..., 2.4265e-26,
           2.7448e-22, 2.1886e-38],
          [6.8552e-31, 4.3157e-36, 2.1954e-40,  ..., 1.7501e-09,
           1.9810e-26, 2.3046e-36],
          ...,
          [2.6991e-39, 1.9612e-35, 3.6632e-34,  ..., 2.9472e-17,
           1.9030e-23, 1.2869e-15],
          [1.7963e-30, 1.0558e-25, 1.4749e-39,  ..., 1.6257e-19,
           5.8711e-35, 3.4140e-41],
          [1.5762e-27, 1.1590e-39, 1.1788e-25,  ..., 1.4536e-31,
           3.1501e-41, 1.8450e-37]]], device='cuda:0'),
 'model_output': tensor([[[ -7.6601,  -0.8689,  -7.2604,  ...,   3.4846,  -5.5560,  10.4001],
          [-10.0051,   4.4050,  -5.8318,  ...,   7.3675,  -6.2918,  -8.7680],
          [  9.9854,  -7.4684,   4.6732,  ...,   4.2836,   3.1180,   9.5783],
          ...,
          [-13.8079, -10.7219,  -7.3332,  ...,  -6.0323, -10.5765,   6.6447],
 

In [27]:
from src.inference.discrete_inference import bayesian_inference, sample_model_output, y, accuracy

In [17]:
bayesian_inference(
    model_input = previous_iteration["model_input"],
    model_output_logits = previous_iteration["model_output"],
    i = torch.tensor([3], device=previous_iteration["model_input"].device),
    n = torch.tensor([190], device=previous_iteration["model_input"].device),
    beta_1 = torch.tensor([4.0], device=previous_iteration["model_input"].device)
)

tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0')

In [20]:
sampled_out = sample_model_output(
    model_output_logits = previous_iteration["model_output"],
)

In [28]:
acc = accuracy(
    i = torch.tensor([3], device=previous_iteration["model_input"].device),
    n = torch.tensor([190], device=previous_iteration["model_input"].device),
    beta_1 = torch.tensor([4.0], device=previous_iteration["model_input"].device),
)

In [30]:
y_res = y(sampled_out, acc)

In [37]:
torch.exp(y_res).isnan().any()

tensor(False, device='cuda:0')

In [39]:
previous_iteration["model_input"].isnan().any()

tensor(False, device='cuda:0')

In [44]:
y_res.shape, previous_iteration["model_input"].shape

(torch.Size([1, 32, 50257]), torch.Size([1, 32, 50257]))

In [47]:
res = torch.exp(y_res) * previous_iteration["model_input"]

In [48]:
res

tensor([[[1.0103e-34, 0.0000e+00, 4.8732e-12,  ..., 1.4212e-33,
          0.0000e+00, 5.4452e-22],
         [0.0000e+00, 3.0360e-33, 0.0000e+00,  ..., 3.7671e-30,
          3.6128e-37, 8.8579e-36],
         [1.4412e-19, 1.7802e-29, 2.8089e-30,  ..., 6.1808e-02,
          1.8880e-28, 0.0000e+00],
         ...,
         [0.0000e+00, 6.7687e-41, 2.0210e-37,  ..., 6.7457e-08,
          3.4779e-23, 8.3339e-25],
         [8.1486e-24, 1.4185e-09, 7.7263e-28,  ..., 1.2398e-03,
          0.0000e+00, 0.0000e+00],
         [3.6134e-34, 1.1812e-21, 9.3908e-01,  ..., 5.1478e-28,
          0.0000e+00, 4.2815e-21]]], device='cuda:0')

In [49]:
res[0][0][1413]

tensor(nan, device='cuda:0')

In [38]:
torch.nonzero(torch.isnan(res))

tensor([[    0,     0,  1413],
        [    0,     1, 46425],
        [    0,     2, 24362],
        [    0,     2, 43902],
        [    0,     4,  3860],
        [    0,     4, 29745],
        [    0,     5, 10265],
        [    0,     5, 29235],
        [    0,     5, 37292],
        [    0,     6, 19952],
        [    0,     6, 41152],
        [    0,     7, 24566],
        [    0,     8,  2811],
        [    0,     8, 23228],
        [    0,     8, 33144],
        [    0,     8, 33913],
        [    0,     9, 10747],
        [    0,     9, 11843],
        [    0,     9, 25087],
        [    0,     9, 42037],
        [    0,     9, 49324],
        [    0,    10, 27499],
        [    0,    12, 46786],
        [    0,    13, 10680],
        [    0,    13, 14675],
        [    0,    13, 15454],
        [    0,    13, 18659],
        [    0,    13, 30122],
        [    0,    13, 50028],
        [    0,    14, 14627],
        [    0,    14, 18296],
        [    0,    15, 38881],
        

In [50]:
torch.exp(y_res[0][0][1413]), previous_iteration["model_input"][0][0][1413]

(tensor(inf, device='cuda:0'), tensor(0., device='cuda:0'))

In [46]:
y_res[0][0][1413] * previous_iteration["model_input"][0][0][1413]

tensor(0., device='cuda:0')

In [53]:
torch.exp(y_res[0][0][1413]) * ( 1 + previous_iteration["model_input"][0][0][1413])

tensor(inf, device='cuda:0')